# Packages installation

In [ ]:
#install packages needed 
!pip install matplotlib
!pip install particles

# Importations

In [1]:
import matplotlib.pyplot as plt 
import numpy as np
import seaborn as sb

# Modules from particles
import particles 
from particles import distributions as dists # Where proba distributions are defined
from particles import state_space_models as ssm # Where state-space-models are defined
from particles.collectors import Moments

# A word about what we are trying to achieve

# Creating our model

## On the use of uniforms to make "choices"

Here, we rely on the "particles" package for the distributions. 
The following is the code that allows to draw a number from a uniform continuous distribution on [0,1]. 
Note that first we $define$ the law (as an object), then we can generate numbers from it.

In [31]:
Uniform = dists.Uniform(a = 0, b = 1)
# Generate one number from a (continuous) uniform(0,1)
x = Uniform.rvs(size = 1)
print(x)

# Equivalently
y = dists.Uniform(a = 0., b = 1.).rvs(size = 1)
print(y)

[0.05535058]
[0.64745081]


In [33]:
# Generate a bunch of uniforms 
vector_x = Uniform.rvs(size = 10000)
#plt.hist(vector_x, density = True, bins = 100)
#plt.show()

# Equivalently 
vector_y = dists.Uniform(a = 0., b = 1.).rvs(size = 10000)
#plt.hist(vector_y, density = True, bins = 100)
#plt.show()

In our case, uniforms have two different uses:
- We can create more complex distributions based on uniforms (see next section for more details)
- We can use them to make **choices**
    
The markov transition kernels for $u_{t}$ consists in a mixture of different possible **moves**. If, for example, $u_{t-1} = 0$, $u_t$ either stays at 0 (with probability $1 - \eta$) or follows an exponential distribution (with probability $\eta$). In our implementation, it is necessary, at each step, to choose between these options.

One way to do it is to generate a uniform $u \sim \mathcal{U}(0,1)$, then to check if $ u < \eta$. If so, $u_{t}$ should follow an exponential, if not, it should stay at 0.

This procedure can be extended to multiple moves. Say we have 3 options, with respective probabilities $\alpha$, $\beta$ and $\gamma$ such that $\alpha + \beta + \gamma = 1$. First we generate $u \sim \mathcal{U}(0,1)$. Then we check:
- $u < \alpha$ ? If so: option 1, if not, continue the loop.
- $u < \alpha + \beta$ ? If so:option 2, if not, continue the loop.
- Given that probabilities sum up to 1, the only possibility left is option 3. 


## About the distributions 

All probability distributions are not available in the package (see the documentation https://particles-sequential-monte-carlo-in-python.readthedocs.io/en/latest/distributions.html). However, it is possible to define **transformations** of distributions and, as such, to create new ones. The two next transformations will prove useful. 

If $X \sim \mathcal{U}(0,1)$ then $-\frac{1}{\lambda} log X \sim \mathcal{E}(\lambda)$ (here, $\lambda$ is the **rate**). 

If $Y \sim \mathcal{G}a(n, \frac{1-p}{p})$ and $ X|y \sim \mathcal{P}(y)$ then $X \sim \mathcal{N}eg(n,p)$  

The following pieces of codes are used to generate such laws using the functions in the package.

In [ ]:
# Exponential with rate zeta 
zeta = 2
exp_zeta = dists.LinearD(dists.LogD(dists.Uniform(a = 0., b = 1.)), a = - 1 / zeta)

In [36]:
# Negative Binomial n,p

# Need to check the parametrization of gammas and NG we are talking about !
# Unsure if that is possible using the pacakge, considering the transforms are linear, not nested.

## Actual definition of the model

In [ ]:
class expression(ssm.StateSpaceModel):
    
    """
    # Work in progress. In the end, we should add default values of the parameters here. 
    default_params = {"eta":, "alpha":, "beta":, "beta_0", "gamma_u":, "gamma_d":,
                     "zeta":, "lamba_u":, "lambda_d": ,
                     "alpha_s":, "kappa_s":}
    """
    
    # Initial distribution of the data (X0)
    def PX0(self): 
    
    
    # Transition kernel 
    def PX(self, t, xp): # Distribution of X_t given X_{t-1} = xp (p = past)
        # Here, X involves two things: u_t (0th index) and s_t (1th index)
        # Therefore, we have to define both 
        
        # For u_t 
        
        uni_x = dists.Uniform(a = 0, b = 1).rvs(size = 1) # Uniform to "choose a move"
        
        if xp[:,0] ==0 :
            if uni <= self.eta:
                # Dirac on xp = 0 
                u_t =  dists.Dirac(loc = 0)
            else:
                # Exponential law with rate zeta 
                u_t = dists.LinearD(dists.LogD(dists.Uniform(a = 0, b = 1)), a = - 1 / zeta)
            
        elif xp[:,0] !=0:
            if uni <= alpha:
                # Dirac on xp
                u_t = dists.Dirac(loc = xp[:, 0])
                
            elif uni <= alpha + beta:
                # Exponential law with rate zeta 
                u_t = dists.LinearD(dists.LogD(dists.Uniform(a = 0, b = 1)), a = - 1 / zeta)
            
            elif uni <= alpha + beta + beta_0:
                # Dirac on 0
                u_t = dists.Dirac(loc = 0)
            
            elif uni <= alpha + beta + beta_0 + gamma_u:
                #
                u_t = 
            
            else:
                #
                u_t = 
                
        #For s_t
        
        uni_s = dists.Uniform(a = 0, b = 1).rvs(size = 1) # Uniform to "choose" a move 
        if uni_s <= alpha_s:
            s_t = dists.Dirac(loc = xp[:,1])
        else:
            s_t = dists.Gamma(a = kappa_s, b = kappa_s)
            
        return IndeProd(u_t, s_t)

            
    # Emission model
    def PY(self, t, xp, x): # Distribution of Y_t given X_t = x (and possibly X_{t-1} =xp)
        
        